In [1]:
from dotenv import load_dotenv
from langchain.embeddings import LlamaCppEmbeddings, OpenAIEmbeddings
from langchain.llms import LlamaCpp
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.vectorstores import FAISS
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from tqdm import tqdm

import os
import pickle

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
llm = LlamaCpp(
    model_path="./backups/llama.cpp/models/llama-2-7b-chat.ggmlv3.q4_K_S.gguf.bin", 
    n_ctx=2048, 
    verbose=False)
llm.client.verbose = False

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./backups/llama.cpp/models/llama-2-7b-chat.ggmlv3.q4_K_S.gguf.bin (version unknown)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:               output_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:                    output.weight q6_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:              blk.0.attn_v.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    6:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:           blk.0.attn_norm.weight f32      [

In [4]:
def tokenizer(text):
    return word_tokenize(text.lower())

In [5]:
queries = {}

idx = None
with open("../data/cisi/CISI.QRY") as f:
    for query in f.read().split(".I"):
        for i, line in enumerate(query.split("\n")):
            if not line:
                continue
            elif i == 0:
                idx = int(line)
                queries[idx] = ""
            elif not line.startswith("."):
                queries[idx] += " " + line

In [6]:
docs_file_path = './backups/openai_embeddings/doc_embeddings.pkl'

with open(docs_file_path, 'rb') as file:
    docs = pickle.load(file)

print("Document embeddings loaded successfully.")

Document embeddings loaded successfully.


In [7]:
docs[1]['body']

'.W\nThis report is an analysis of 6300 acts of use\nin 104 technical libraries in the United Kingdom.\nLibrary use is only one aspect of the wider pattern of\ninformation use.  Information transfer in libraries is\nrestricted to the use of documents.  It takes no\naccount of documents used outside the library, still\nless of information transferred orally from person\nto person.  The library acts as a channel in only a\nproportion of the situations in which information is\ntransferred.\nTaking technical information transfer as a whole,\nthere is no doubt that this proportion is not the\nmajor one.  There are users of technical information -\nparticularly in technology rather than science -\nwho visit libraries rarely if at all, relying on desk\ncollections of handbooks, current periodicals and personal\ncontact with their colleagues and with people in other\norganizations.  Even regular library users also receive\ninformation in other ways.\n'

In [8]:
rel_set = {}
with open("../data/cisi/CISI.REL") as f:
    for l in f.readlines():
        qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]
        doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]

        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)

In [9]:
# Load index from file
loaded_faiss_vs = FAISS.load_local(
    folder_path="./backups/faiss/",
    embeddings=OpenAIEmbeddings())

retriever = loaded_faiss_vs.as_retriever(search_kwargs={'k': 10})

# Define the RAG pipeline
template = """
Answer the question or Explain the topic given this additional context: {context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [10]:
def format_docs(_docs):
    res = ""
    for doc in _docs:
        if int(doc.page_content) in docs:
            res += docs[int(doc.page_content)]["body"]
            res += "\n\n"
    return res[:1800]

In [11]:
chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()} 
         | prompt 
         | llm 
         | StrOutputParser())


In [12]:
chain.invoke(queries[90])

'Explain the concept of paradigms and how it relates to citation analysis.  \n'

In [13]:
#### API CALL WARNING #####

rag_responses = {}

# Run RAG pipeline for every question
for query_id in tqdm(rel_set.keys(), desc = 'Asking Queries to Local Llama with RAG'):
    query_text = queries[int(query_id)]
    response = chain.invoke(query_text)
    rag_responses[query_id] = response

Asking Queries to Local Llama with RAG:   0%|                         | 0/76 [00:15<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Save RAG responses to a pickle file
rag_responses_file_path = './backups/llama_with_rag_responses.pkl'
with open(rag_responses_file_path, 'wb') as file:
    pickle.dump(rag_responses, file)

print(f"RAG responses saved to {rag_responses_file_path}")

In [14]:
llama_with_rag_responses_file_path = './backups/llama_with_rag_responses.pkl'
with open(llama_with_rag_responses_file_path, 'rb') as file:
    rag_responses = pickle.load(file)

In [15]:
 # Sanity
rag_responses['90']

'Paradigm: \nA consensual structure of concepts in a field.  '

In [16]:
# Implement BLEU evaluation function
def compute_bleu(references, candidate):
    smoothing = SmoothingFunction().method0
    return sentence_bleu(references, candidate, smoothing_function=smoothing)

In [17]:
# Implement ROUGE evaluation function
def compute_rouge(references, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    total_score = 0

    # Compute ROUGE for each reference
    for reference in references:
        scores = scorer.score(reference, candidate)
        total_score += scores['rouge1'].fmeasure

    # Calculate average score
    average_score = total_score / len(references)
    return average_score

In [18]:
# Evaluate BLEU and ROUGE for each query

K = 30 # Number of most relevant docs to consider for scoring performance
total_bleu_score = 0.0
total_rouge_score = 0.0
num_queries = 0

for query_id, relevant_docs in rel_set.items():
    query_text = queries[int(query_id)]
    response = rag_responses[query_id]

    # print(query_id, query_text, "\n\nResponse:\n", response, "\nTopmost relevant Doc:\n", docs[int(relevant_docs[0])], "\n======\n")
    
    # Evaluate using BLEU
    bleu_score = compute_bleu([docs[int(id)]['body'] for id in relevant_docs[:K]], response)
    total_bleu_score += bleu_score

    # Evaluate using ROUGE
    rouge_score = compute_rouge([docs[int(id)]['body'] for id in relevant_docs[:K]], response)
    total_rouge_score += rouge_score

    num_queries += 1
    # if num_queries == 10:
    #     break

# Calculate mean scores
mean_bleu_score = total_bleu_score / num_queries
mean_rouge_score = total_rouge_score / num_queries

print(f"Mean BLEU Score: {mean_bleu_score:.4f}")
print(f"Mean ROUGE Score: {mean_rouge_score:.4f}")

Mean BLEU Score: 0.7047
Mean ROUGE Score: 0.1826
